# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
nBlack = sum(data.race=='b')
bCalled = sum(data[data.race=='b'].call)

nWhite = sum(data.race=='w')
wCalled = sum(data[data.race=='w'].call)

# number of callbacks for black-sounding names
print(nBlack, bCalled)
print(nWhite, wCalled)

2435 157.0
2435 235.0


In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


1.A permutation test would be appropriate for this problem since we are testing two categories for a boolean response. CLT does apply because there is a large sample size.

2.Replicating the study assuming race was irrelevant (This is the null hypothesis)
Explicitly, I will find the probability that permuted replications of the data have a ratio of "black" people (after the permutation, where races were mixed together) receiving a call back that is at least as small as the observed ratio.

In [5]:
#Borrowing this from the datacamp course
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1,data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1,perm_sample_2)

    return perm_replicates

#The statistic we will use
def blackFrac(black,white):
    return np.sum(black)/len(black)

In [34]:
#The null hypothesis is any random permutation of our data, we may find that the "black" partition can have as low of a call-rate as our sample data
replicates = draw_perm_reps(data.loc[data.race=='b','call'],data.loc[data.race=='w','call'],blackFrac,10000)
p = np.sum(replicates<=bCalled/nBlack)/len(replicates)
print(np.sum(replicates<=bCalled/nBlack))
print('p:',p)

1
p: 0.0001


The above computation shows that, when the white and black populations were mixed in the sample, there was only one instance where the callback rate for the "black" population was as low as the callback rate for the actual black population in the sample. This shows very strong statistical grounds <b>(confidence level of .9999)</b> for the idea that whether or not someone is called for a job interview may be related to the ethnicity portrayed by their name.

Let's see if there seems to be any relation with the boolean field, military.

In [6]:
nMilitary = sum(data.military==1)
militaryCalled = sum(data.loc[data.military==1,'call'])

nNonMilitary = sum(data.military==0)
nonMilitaryCalled = sum(data.loc[data.military==0,'call'])

print(militaryCalled/nMilitary,nonMilitaryCalled/nNonMilitary)

0.0634249471459 0.0823288605868


Looks like non-military applicants had a slight advantage as well. Let's see if the same occurs after permutations. The null hypothesis here is the fact that military service was unrelated to callbacks, whereas the alternate hypothesis is the fact that non-military applicants had an advantage over the military applicants.

In [7]:
#I will use the same value function as before, just pretending non-military is the white (advantaged) group and the military is the black (disadvantaged) group
replicates = draw_perm_reps(data.loc[data.military==1,'call'],data.loc[data.military==0,'call'],blackFrac,10000)
p = np.sum(replicates<=militaryCalled/nMilitary)/len(replicates)
print(np.sum(replicates<=militaryCalled/nMilitary))
print('p:',p)

837
p: 0.0837


The high p-value (>.05) means that there were a considerable number of permutations that consisted of at least as low of a callback rate in the "military" group as was observed in the sample. This means that the low callback rate in the observed data may have been because of the low sample size, since there were far more non-military applicants observed.

Another factor that should account for whether or not a person gets called back is their years of experience, so lets look at how years of experience affected the callback rates.

In [16]:
blackDf = data.loc[data.race == 'b']
whiteDf = data.loc[data.race == 'w']

print('Black avg years of exp: ',blackDf.yearsexp.mean())
print('White avg years of exp: ',whiteDf.yearsexp.mean())

print('Median black years of exp:',blackDf.yearsexp.median())
print('Median white years of exp:',whiteDf.yearsexp.median())

Black avg years of exp:  7.8295687885
White avg years of exp:  7.85626283368
Median black years of exp: 6.0
Median white years of exp: 6.0


Looks like the black and white applicants both had very similar years of experience, but how did years of experience effect the callback rate?

In [27]:
print('Average years of experience for callback:',data.loc[data.call==1,'yearsexp'].mean())
print('Average years of experience for white callback:',whiteDf.loc[whiteDf.call==1,'yearsexp'].mean())
print('Average years of experience for black callback:',blackDf.loc[blackDf.call==1,'yearsexp'].mean())

print('Median years of experience for callback:',data.loc[data.call==1,'yearsexp'].median())
print('Median years of experience for white callback:',whiteDf.loc[whiteDf.call==1,'yearsexp'].median())
print('Median years of experience for black callback:',blackDf.loc[blackDf.call==1,'yearsexp'].median())

print('Average years of experience for no callback:',data.loc[data.call==0,'yearsexp'].mean())
print('Average years of experience for no white callback:',whiteDf.loc[whiteDf.call==0,'yearsexp'].mean())
print('Average years of experience for no black callback:',blackDf.loc[blackDf.call==0,'yearsexp'].mean())

Average years of experience for callback: 8.89030612245
Average years of experience for white callback: 8.7829787234
Average years of experience for black callback: 9.05095541401
Median years of experience for callback: 7.0
Median years of experience for white callback: 7.0
Median years of experience for black callback: 7.0
Average years of experience for no callback: 7.75122822689
Average years of experience for no white callback: 7.75727272727
Average years of experience for no black callback: 7.74539069359


It looks like black applicants did need a slightly higher number of years of experience to receive a call. But let's test that hypothesis

In [25]:
def TTestForEqualMeans(sample1,sample2):
    diffOfMeans = abs(sample1.mean()-sample2.mean())

    se = math.sqrt(((sample1.std()**2)/sample1.size) + ((sample2.std()**2)/sample2.size))

    t = diffOfMeans/se
    p = stats.t.cdf(t,sample2.size+sample1.size-2)

    print('t:',t,'p:',p)

In [26]:
import math

#Null hypothesis: The average number of years of experience that white and black applicants that received a call back are the same
whiteCalled = whiteDf.loc[whiteDf.call==1,'yearsexp']
blackCalled = blackDf.loc[blackDf.call==1,'yearsexp']

TTestForEqualMeans(blackCalled,whiteCalled)

t: 0.459752359087 p: 0.67702491693


With such a high p-value, we cannot reject the null hypothesis that these averages are the same. The difference between them are not significant.

Judging by the features I've studied, it looks like applicants with similar credentials may not be receiving calls back based on the ethnicity portrayed by their name.